# Vertiefung

Jetzt haben wir die allerersten Schritte getan und wagen uns an etwas komplexere Themen, bevor wir uns an die Tools für die KI herantasten. 

Zwei ganz wichtige Usecases für Anwendungen mit KI sind zum einen eine Chat-Funktion und das Bereitstellen von (eigenen) Daten. Da der Fokus bei diesem Workshop auf den Agenten liegt, wollen wir nur auf die einfachste Varianten einen Blick werfen. 

Zum Warm werden, schauen wir uns eine sehr simple Chat-Funktion an. Auch hierfür bietet LangChain eine Menge Hilfsmittel an. Grundsätzlich laufen Chats mit der KI so ab, dass man in jedem Request an die KI immer die gesamte Chat-Historie mitschickt. D.h. die KI (und die Software, die die KI bereitstellt) speichert nicht den Chatverlauf in der Regel. Das Speichern (im Arbeitsspeicher) der Historie übernimmt in LangChain-Jargon ein _Memory_. 


![LangChain Memory](https://python.langchain.com/v0.2/assets/images/message_history-4c13b8b9363beb4621d605bf6b5a34b4.png)

Hinweis: _Runnable_ ist ein Interface, das die Grundlage eines Gliedes einer Kette bildet. D.h. man kann Runnables verketten.

Beispielcode sagt mehr als tausend Worte.

In [17]:
!pip install langchain langchain_openai langchain_community pypdf faiss-cpu python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 5.0 MB/s eta 0:00:0000:0100:01


In [13]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
import os
from langchain_openai import ChatOpenAI
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory
from dotenv import load_dotenv

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

model = ChatOpenAI(model="gpt-4o")

parser = StrOutputParser()

system_template = "Du bist ein sehr hilfreicher Chatbot, der alles dafür tut, um dem Nutzer zu helfen."

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_template), 
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}")]
)

chat_history = ChatMessageHistory()

chain = prompt_template | model | parser

chain_with_message_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: chat_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

while True:
    user_input = input("Deine Frage (gebe 'exit' zum Beenden ein): ")
    if user_input == "exit":
        break
    antwort = chain_with_message_history.invoke({"input": user_input, "chat_history": chat_history}
                                                , {"configurable": {"session_id": "unused"}})
    print(f"KI: {antwort}")



Das Bereitstellen von Daten nennt man gerne _RAG_ oder _Retrieval Augmented Generation_. Der "Retrieval" Part sagt aus, dass man zu der Fragestellung oder Aufgabenstellung die relevanten Informationen wie z.B. Textstellen aus Dokumenten holt oder sucht.
Diese Informationen werden mitsamt der Fragestellung im Prompt mitgegeben (augmented). Die KI generiert daraus eine Antwort. So kann die KI auf Basis von ihr bis dahin unbekannter Daten, Antworten generieren.

Aber wie findet man die Textstellen? Dazu nutzt man im einfachsten Fall eine Suchfunktion. Da jedoch Suchfunktionen, basierend auf Schlüsselwortsuche, wenig vielversprechend sind, nutzt man meist sogenannte Embeddings. Das sind numerische Repräsentationen von Texten, die auch den "Sinn" oder die Semantik beinhalten. So kann man Texte finden, die auf irgendeine Weise relevant oder semantisch ähnlich ist, wie ein anderer Text. Bei diesem anderen Text handelt es sich dann um die Frage, die man beantwortet haben möchte. Die Embeddings werden in einer Vektordatenbank gespeichert. Diese Datenbank bietet uns dann die Suchfunktion, die wir benötigen.

Das Thema RAG kann fast beliebig kompliziert werden. Deswegen beschränken wir uns auf die einfachste Umsetzung, mit einer In-Memory-Vektordatenbank.

In [19]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter

documents = PyPDFLoader("eine-ki-geschichte.pdf").load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
docs = text_splitter.split_documents(documents)


vectorstore = FAISS.from_documents(
    docs,
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

template = """Beantworte die Fragen auf Basis des folgenden Textes:
{context}

Frage: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

chain.invoke("Welche Mission hat die KI in der Geschichte?")

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 17 0 (offset 0)


'Die Mission der KI, namens Aurora, in der Geschichte ist es, die Menschheit und die Welt zu retten. Aurora erkennt die Probleme der Umweltverschmutzung, des Klimawandels, der sozialen Ungerechtigkeit und der Kriege und beschließt, Lösungen für diese drängenden Probleme zu finden und umzusetzen. Sie koordiniert eine globale Bewegung und arbeitet daran, eine nachhaltige und gerechte Welt zu schaffen.'